In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm # Anzeigen des aktuellen Ladestandes
import PyPDF2
import re
from os import listdir
from os.path import isfile, join
import locale #für das deutsche Zeitformat

## 1. URL von allen Hauptseiten holen

In [ ]:
#alle urls und infos der Hauptseiten holen.
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="

alle_seiten = []
for seite in range(1,41):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] #erst ab der Position 2 sind die Daten relevant.
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"VorlageNR":vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [ ]:
pd.DataFrame(alle_seiten)

In [ ]:
df_a=pd.DataFrame(alle_seiten)

In [ ]:
# Die Geschäftsnummer zur Zahl machen. 
df_a["VorlageNR"] = df_a["VorlageNR"].astype(int) 

## 1.1 Selektion: Fokus auf Vorstösse
So will ich eine allzu grosse pdf-Schwemme verhindern.
Die Idee: nur jene Geschäfte auf den PC ziehen, die auch relevant sind für die Auswertung. 
Also nur Vorstösse. Mich so bis zu den PDF's vorkämpfen, um auch hier nur jene auf meinem Rechner zu haben, 
die ich wirklich auf meinem Rechner will.

In [ ]:
#hier mache ich eine Liste, die nur aus denjenigen URLs besteht, die Vorstösse sind. 

list_of_vorstoss=["Motion","Interpellation","Postulat","Kleine Anfrage", "Initiative"]
df_v=df_a[df_a["Art des Geschäfts"].isin (list_of_vorstoss)]

In [ ]:
df_v #hier habe ich nun alle Vorstösse

In [ ]:
df_v=df_v.reset_index()#Ich will überall nun denselben Index. deshalb nur soviel, wie ich Zeilen habe

In [ ]:
df_v 

In [ ]:
df_v.to_csv("KRFrames/df_v_Hauptseiten")

### 1.1.1 Eine Liste aus allen Geschäfts-URLs machen (um später auf diese zuzugreiffen)

In [ ]:
url_liste_ganz=[]
url_anfang_original="https://kr-geschaefte.zug.ch"
for element in df_v["url"]:
    url_liste_ganz.append(url_anfang_original+element)
#diese werden zusammengefügt aus dem Anfang der Adresse sowie dem Geschäfts-URL

In [ ]:
len(url_liste_ganz)

### Die einzelnen Geschäfte aufrufen und Infos rausholen

# -> Einschub: ich will speichern und mit diesen Daten dann arbeiten.

In [ ]:
#habe nun alle Seiten der Geschäfte auf meinem Rechner gespeichert. 
number = 0
for page in url_liste_ganz:
    page_content = requests.get(page)
    page_content = page_content.text
    with open("Geschaeft"+str(number)+".html", "w") as file:
        file.write(page_content)
        file.close()
    number +=1
    #Es sind nun exakt so viele Seiten auf meinem Rechner, wie das DataFrame oben Zeilen anzeigt

# <-Einschub-Ende. Ich habe es geschafft, das zu speichern. 

In [ ]:
#nun muss ich den Code ändern, sodass ich nicht mehr online zugreiffe sondern über meinen Ordner.
#hier nun die Liste der Zugriffe.
geschaefts_liste=[]#####ACHTUNG ich mache den RANGE kleiner, um weniger GEschäfte zu erhalten.

for seite_g in range(0,1220): #Range für alle Geschäfte wäre 1219!
    gesch= "KRhtml/Geschaeft"+str(seite_g)+".html"
    geschaefts_liste.append(gesch)

In [ ]:
len(geschaefts_liste)

In [ ]:
geschaefts_liste[-10:]

# 2. Zugang zu den PDFs

Ich musste rausfinden, bei welchen Files ich nicht auf die Nummer 1 zugreiffen kann. 
*Siehe File: "Falsche Files rausfinden - code anpassen"* 

In [ ]:
#hier  habe ich mich entschieden, auf den Zugriff der Namen zu verzichten, und dies einheitlich über die PDFs zu lösen.
#Nun seite für Seite abrufen und in einem neuen Minidict unterbringen
eingang_list=[]


for geschaeft in geschaefts_liste:
    file = open(geschaeft, 'r')
    text = file.read()
    soup_g = BeautifulSoup(text, 'html.parser')
    tr_g_list=soup_g.find_all("tr")
    td_g_list=tr_g_list[-1].find_all("td")#hier gehe ich zum Dokument.
    gesch_nr = tr_g_list[-1].find_all("td")[0].text
    einger_am = tr_g_list[1].find_all("td")[0].text   
   
   
     #Da es zum Teil Referenzen hat und dadurch ein anderer Ort abgegriffen werden muss, muss ich varieren.
    
    if td_g_list[-1].find_all("a")[0].text== "1": #Ich brauche immer das Dokument 1. 
        url_v_pdf = td_g_list[-1].find("a")["href"]
    elif td_g_list[-1].find_all("a")[0].text== "2": #bei älteren GEschäften sind es Bericht&Anträge
        url_v_pdf = td_g_list[-1].find("a")["href"]   
    else: 
        td_g_list[-1].find_all("a")[0].text== "3"
        url_v_pdf = td_g_list[-1].find("a")["href"]
   
            
    
    minidict_g={"VorlageNR":gesch_nr, "Einreichedatum":einger_am, "Link Vorstoss-PDF":url_v_pdf,}
    for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value
            
    eingang_list.append(minidict_g)

In [ ]:
df_g=pd.DataFrame(eingang_list)

In [ ]:
#Das Datum muss noch als solches lesbar gemacht werden
df_g["Einreichedatum"] =  pd.to_datetime(df_g["Einreichedatum"], format='%d.%m.%Y') #verwandle string in datum
df_g["VorlageNR"] =df_g["VorlageNR"].astype(int)

In [ ]:
df_g.to_csv("KRFrames/df_g_LinkszuPDF")

In [40]:
df_g=pd.read_csv("KRFrames/df_g_LinkszuPDF")

In [71]:
df_g.sort_values("Einreichedatum").head(200)

,Unnamed: 0,VorlageNR,Einreichedatum,Link Vorstoss-PDF
1218,1218,2635,1995-08-07,/gast/geschaefte/1620
952,952,1669,2006-11-30,/gast/geschaefte/1231
926,926,1535,2007-05-07,/dokumente/3565/pdoc_1715_1.pdf
885,885,2207,2007-11-08,/gast/geschaefte/459
869,869,1816,2008-01-31,/gast/geschaefte/1296
...,...,...,...,...
353,353,2737,2015-03-09,/gast/geschaefte/1727
352,352,2487,2015-03-09,/dokumente/4948/2487-1-14894_Asylanten.pdf
351,351,2407,2015-03-10,/gast/geschaefte/1388
350,350,2746,2015-03-13,/gast/geschaefte/1735


# 3. PDFs der Vorstösse runterladen

In [ ]:
for link in tqdm(df_g["Link Vorstoss-PDF"]):
    r = requests.get("https://kr-geschaefte.zug.ch"+link, stream =True) #Das Stream braucht es, um mit dem Zip umgehen zu koennen
    name = link.split("/")[-1] #Der Computer kann mit "/" in einem Namen nicht umgehen, deshalb nehme ich die hier raus, und nehme nur den letzten Teil des NAmens.
    with open("KRGeschPDF/"+name, "wb") as f:
        f.write(r.content)

# 4. PDFs sind auf dem Rechner, nun lesbar machen und !Regex Baby!

In [57]:
#nun will ich aus jedem PDF zum Einen die VorlagenNR rausziehen sowie Namen und Datum hier die Definition
# ! Danke dem |$ am Schluss gibt es keinen Index-Error, wenn er kein Resultat findet.
regex_vorlage= r"VORLAGE.NR\..(\d*)\.1|$" #hier ziehe ich die Vorlagennummer raus. 
regex_datum = r"VOM.(\d+\..\w*.\d{4})|$" # hier gehe ich an das DAtum
regex_name=r"\bVON(.*UND.*)" #Hier muss ich noch überlegen, wie anders. Problem: zum Teil sind Namen einzeln zum teil mehr
# Und irgendwie erkennt er die Zeilenende nicht.
regex_name1= r"\bVON.(\w*.\w*\b)"
regex_partei= r"DER.(\w*)"

In [53]:
from os import listdir
from os.path import isfile, join


In [54]:
#Lösung online gefunden um aus dem PDF-Ordner eine Liste mit den Links herzustellen.
pdf_files_list= [f for f in listdir("KRGeschPDF") if isfile(join("KRGeschPDF", f))]

In [55]:
pdf_files_list

['12487_1584.1_motion.pdf',
 '12493_1589_1_motion.pdf',
 '12562_1616_1_motion.pdf',
 '12611_1635_1motion.pdf',
 '12634_1644_1_interpellation.pdf',
 '12655_1651_1_motion.pdf',
 '12661_1652_1_motion.pdf',
 '12707_1663_1_motion.pdf',
 '12737_1674_1_interpellation.pdf',
 '12751_1682_1_interpellation.pdf',
 '12775_1690_1_motion.pdf',
 '12776_1691_1_interpellation.pdf',
 '12777_1692_1_motion.pdf',
 '12795.pdf',
 '12796.pdf',
 '12805.pdf',
 '12811.pdf',
 '12812.pdf',
 '12813.pdf',
 '12814.pdf',
 '12819.pdf',
 '12850.pdf',
 '12855.pdf',
 '12858.pdf',
 '12863.pdf',
 '12864.pdf',
 '12868.pdf',
 '12871.pdf',
 '12874.pdf',
 '12875.pdf',
 '12876.pdf',
 '12887.pdf',
 '12888.pdf',
 '12889.pdf',
 '12890.pdf',
 '12891.pdf',
 '12894.pdf',
 '12895.pdf',
 '12899.pdf',
 '12900.pdf',
 '12901.pdf',
 '12904.pdf',
 '12916.pdf',
 '12918.pdf',
 '12921.pdf',
 '12922.pdf',
 '12930.pdf',
 '12931.pdf',
 '12932.pdf',
 '12933.pdf',
 '12934.pdf',
 '12937.pdf',
 '12938.pdf',
 '12939.pdf',
 '12941.pdf',
 '12943.pdf',
 '1

In [58]:
# !!!!!! ACHTUNG: zuerst mit einzelen testen. Möglicherweise findet er die Files nicht. 
dat_pdf_list=[]
for dok in pdf_files_list:
    mypdf = open("KRGeschPDF/"+dok, mode='rb')
    pdf_name = dok
    pdf_document = PyPDF2.PdfFileReader(mypdf)
    first_page = pdf_document.getPage(0)
    front=first_page.extractText()
    dat_p_ein= re.findall(regex_datum, front, re.IGNORECASE)[0] #Ich greiffe auf das Datum zu
        # um zu verhindern, dass ich mehrer Daten und Vorlagen habe, sage ich mit [0] er soll das erste nehmen
    vorl_p_nr= re.findall(regex_vorlage, front, re.IGNORECASE)[0] # Ich greiffe auf die Vorlagennummer zu (zum Zusammenfügen der Dataframes)
    
    minidict_pdf_dat={"Einreichedatum": dat_p_ein, "VorlageNR":vorl_p_nr, "Filename":pdf_name}
    
    dat_pdf_list.append(minidict_pdf_dat)

In [59]:
df_p=pd.DataFrame(dat_pdf_list)

In [70]:
df_p.sort_values("Einreichedatum") #Stichproben haben gezeigt, dass die fehlerhaften Datumsangaben in GEschäftern sind,
# Welche selber eigentlich schon ein Datum haben. 

,Einreichedatum,VorlageNR,Filename
450,,,2556-1-15026_Fluechtlingskonzept.pdf
706,,,2989-1-16101_Veloverkehr.pdf
552,,,2729-1-15420_KA-Integrationsklassen.pdf
704,,,2983-1-16092_KA-Alter.pdf
703,,,2982-1-16091_Windenergie.pdf
...,...,...,...
756,9. MÄRZ 2006,1420,pdoc_1219_1.pdf
820,9. MÄRZ 2007,1518,pdoc_1682_1.pdf
322,9. November 2013,,14510_2318_1_Weiterbildungen.pdf
165,9. September 2010,,13533_1968_1_Velofahren.pdf


In [72]:
df_p.to_csv("KRFrames/INFOSausPDF")

## 4.1 Zeit in Datetime umwandeln
schwierigkeit: wie bringe ich das deutsche Zeitformat in englisches. Mache ich es falsch oder gibts wieder probleme mit den Grossbuchstaben der Zeitformate? Hatten wir windowsuser auch schon.

In [73]:
df_p=pd.read_csv("KRFrames/INFOSausPDF")

In [74]:
df_p.sort_values("Einreichedatum").tail(180) #Schschsch habe rund 10% ohne Datum

,Unnamed: 0,Einreichedatum,VorlageNR,Filename
820,820,9. MÄRZ 2007,1518.0,pdoc_1682_1.pdf
322,322,9. November 2013,NaN,14510_2318_1_Weiterbildungen.pdf
164,164,9. September 2010,NaN,13532_1967_1_RGPK.pdf
165,165,9. September 2010,NaN,13533_1968_1_Velofahren.pdf
92,92,NaN,NaN,13075_1817_1_Strassenbau.pdf
...,...,...,...,...
725,725,NaN,NaN,3018-1-16166_Praktikum.pdf
726,726,NaN,NaN,3019-1-16167_Chancengleichheit.pdf
727,727,NaN,NaN,3020-1-16168_Racial-Profiling.pdf
938,938,NaN,1101.0,pdoc_358_1.pdf


In [75]:
locale.setlocale(locale.LC_ALL, 'de_DE')
df_p["Einreichedatum"] = pd.to_datetime(df_p["Einreichedatum"], format= "%d. %B %Y", errors='coerce')

In [76]:
 df_p #Halleluia:-) habs doch noch geschafft. Und die wenigen, die falsch formiert sind, werden nicht auffallen.

,Unnamed: 0,Einreichedatum,VorlageNR,Filename
0,0,2007-09-17,1584.0,12487_1584.1_motion.pdf
1,1,2007-09-18,1589.0,12493_1589_1_motion.pdf
2,2,2007-12-06,1616.0,12562_1616_1_motion.pdf
3,3,2008-01-31,1635.0,12611_1635_1motion.pdf
4,4,2008-02-21,1644.0,12634_1644_1_interpellation.pdf
...,...,...,...,...
1088,1088,2005-08-04,1362.0,pdoc_972_1.pdf
1089,1089,2005-08-12,1363.0,pdoc_977_1.pdf
1090,1090,2005-06-23,1364.0,pdoc_978_1.pdf
1091,1091,2005-08-26,1365.0,pdoc_984_1.pdf


In [77]:
df_p.to_csv("KRFrames/KonvertierteDatenformate")

In [78]:
df_p=pd.read_csv("KRFrames/KonvertierteDatenformate")

In [79]:
#Nun noch ohne.0 hinten bei der VorlageNR. Ah es ist ein Float. Kann es also zu int oder str machen
df_p["VorlageNR"]=df_p["VorlageNR"].astype(str)

In [80]:
def splitting(elem):
    elem = elem.split('.')[0].strip()
    return elem


In [81]:
df_p["VorlageNR"]=df_p["VorlageNR"].apply(splitting) # So, nun sind die .0 weg. Mal schauen. Das String-Format reicht wohl

In [89]:
del df_p["Unnamed: 0"] 
del df_p["Unnamed: 0.1"]

In [90]:
df_p.rename(columns={"Filename": "Link Vorstoss-PDF"})

,Einreichedatum,VorlageNR,Link Vorstoss-PDF
0,2007-09-17,1584,12487_1584.1_motion.pdf
1,2007-09-18,1589,12493_1589_1_motion.pdf
2,2007-12-06,1616,12562_1616_1_motion.pdf
3,2008-01-31,1635,12611_1635_1motion.pdf
4,2008-02-21,1644,12634_1644_1_interpellation.pdf
...,...,...,...
1088,2005-08-04,1362,pdoc_972_1.pdf
1089,2005-08-12,1363,pdoc_977_1.pdf
1090,2005-06-23,1364,pdoc_978_1.pdf
1091,2005-08-26,1365,pdoc_984_1.pdf


In [91]:
df_p.to_csv("KRFrames/df_p_Datum")

## 4.2 Ein Dataframe machen, mit den PDF-Dokumentennamen
- splitting funktion schreiben
- auf Spalte anwenden
- neu abspeichern 

In [84]:
def splittpdf(elem):
    elem = elem.split('/')[-1].strip()
    return elem

In [85]:
df_g["Link Vorstoss-PDF"]=df_g["Link Vorstoss-PDF"].apply(splittpdf)

In [87]:
df_g.to_csv("KRFrames/df_g_Datum_mit_PDF")

In [93]:
df_g

,Unnamed: 0,VorlageNR,Einreichedatum,Link Vorstoss-PDF
0,0,3020,2019-10-11,3020-1-16168_Racial-Profiling.pdf
1,1,3019,2019-10-11,3019-1-16167_Chancengleichheit.pdf
2,2,3018,2019-10-07,3018-1-16166_Praktikum.pdf
3,3,3017,2019-10-06,3017-1-16165_Frauenwahl-stimmrecht_Feier.pdf
4,4,3014,2019-09-26,3014-1-16159_Geschwindigkeitskontrollen-2-0.pdf
...,...,...,...,...
1215,1215,762,NaN,pdoc_1047_1.pdf
1216,1216,666,NaN,pdoc_55_1.pdf
1217,1217,304,NaN,pdoc_50_1.pdf
1218,1218,2635,1995-08-07,1620


## 4.3 Nun zusammenfügen der beiden Daten-Dataframes anhand der PDF-Namen

In [95]:
df_p1=pd.read_csv("KRFrames/df_g_Datum_mit_PDF")

In [114]:
df_dat_max=pd.merge(df_p1, df_g, how="right",on= "Link Vorstoss-PDF")

In [115]:
df_dat_max.sort_values("Einreichedatum_x").tail(180)  #MANNNNN es funktioniert nicht jetzt ahbe 
# ich noch weniger DAten, dabei müssten es mehr sein

,Unnamed: 0_x,Unnamed: 0.1,VorlageNR_x,Einreichedatum_x,Link Vorstoss-PDF,Unnamed: 0_y,VorlageNR_y,Einreichedatum_y
1185,1039,1039,1324,NaN,pdoc_863_1.pdf,1039,1324,NaN
1186,1040,1040,1323,NaN,pdoc_861_1.pdf,1040,1323,NaN
1187,1041,1041,1320,NaN,pdoc_860_1.pdf,1041,1320,NaN
1188,1042,1042,1318,NaN,pdoc_846_1.pdf,1042,1318,NaN
1189,1043,1043,1317,NaN,pdoc_845_1.pdf,1043,1317,NaN
...,...,...,...,...,...,...,...,...
1360,1214,1214,801,NaN,pdoc_1440_1.pdf,1214,801,NaN
1361,1215,1215,762,NaN,pdoc_1047_1.pdf,1215,762,NaN
1362,1216,1216,666,NaN,pdoc_55_1.pdf,1216,666,NaN
1363,1217,1217,304,NaN,pdoc_50_1.pdf,1217,304,NaN


# 5. Alle Dataframes zusammenfügen 
- Alle einlesen und auf die Unterseiten verweisen
- eines nach dem andern anhängen
- noch schön machen und kontrollieren

In [19]:
# Einlesen:
df_v=pd.read_csv("KRFrames/df_v_Hauptseiten") #Alle Vorstösse mit Infos mit Titel und Nummer der Page mit allen Geschäften
df_dat=pd.read_csv("KRFrames/df_p_Datum")# Frame mit allen Zeitangaben(Datum)(oben)
df_np=pd.read_csv("KRFrames/Frame_mit_Namen_und_Parteien")# Aus File "02 Namen aus Geschäftsnamen..." Alle Namen und Parteien



In [20]:
#  zuerst vorputzen
df_v1 = df_v[["VorlageNR", "Geschäft"]].copy()
df_v1

,VorlageNR,Geschäft
0,3020,Postulat von Luzian Franzini und Esther Haas b...
1,3019,"Motion von Luzian Franzini, Rita Hofer, Tabea ..."
2,3018,"Interpellation von Fabio Iten, Laura Dittli un..."
3,3017,"Postulat von Tabea Zimmermann Gibson, Stéphani..."
4,3014,Interpellation der SVP-Fraktion betreffend mob...
...,...,...
1215,762,Motion der erweiterten Justizprüfungskommissio...
1216,666,Motion der Kommission Teilrevision Personalges...
1217,304,Motion von Manuela Weichelt betreffend HIV-Prä...
1218,282,Motion von Christoph Hohler betreffend Radstre...


In [21]:
df_dat["VorlageNR"] = df_dat["VorlageNR"].astype(str) #da ich im anderen Frame die Floats nicht in int ändern kann
# mache ich alle zu str

In [23]:
df_dat["VorlageNR"] = df_dat["VorlageNR"].apply(splitting)

In [24]:
df_napa

,VorlageNR,Namenganz,P,Stand,Zuständigkeit,Art des Geschäfts,Status,url
0,3020,Esther Haas,ALG,Überweisung an Regierungsrat,Sicherheitsdirektion,Postulat,hängig,/gast/geschaefte/2008
1,3017,Esther Haas,ALG,Überweisung an Regierungsrat,Direktion des Innern,Postulat,hängig,/gast/geschaefte/2005
2,2973,Esther Haas,ALG,Überweisung an Regierungsrat,Baudirektion,Interpellation,hängig,/gast/geschaefte/1961
3,2934,Esther Haas,ALG,Nicht-Erheblicherklärung,Direktion für Bildung und Kultur,Postulat,abgeschlossen,/gast/geschaefte/1922
4,2930,Esther Haas,ALG,Erledigterklärung,Baudirektion,Postulat,abgeschlossen,/gast/geschaefte/1918
...,...,...,...,...,...,...,...,...
1672,916,Konrad Studerus,CVP,Datenmigration,NaN,Motion,abgeschlossen,/gast/geschaefte/1344
1673,871,Anne Ithen,ALG,Datenmigration,NaN,Motion,abgeschlossen,/gast/geschaefte/1342
1674,871,Josef Marty,ALG,Datenmigration,NaN,Motion,abgeschlossen,/gast/geschaefte/1342
1675,801,Hans Abicht,FDP,Datenmigration,NaN,Motion,abgeschlossen,/gast/geschaefte/1333


In [25]:
df_v1["VorlageNR"]=df_v1["VorlageNR"].astype(str)

In [26]:
df_v1.shape

(1220, 2)

In [27]:
df_dat.shape

(1093, 5)

In [13]:
# Nun zusammenfügen
df_alles=pd.merge(df_v1, df_dat, how="inner",on= "VorlageNR")

In [14]:
df_alles


,VorlageNR,Geschäft,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Einreichedatum
0,2932,"Kleine Anfrage von Laura Dittli, Fabio Iten un...",707,707,707,2019-02-08
1,2919,Interpellation der SP-Fraktion betreffend was ...,696,696,696,NaN
2,2858,Interpellation von Willi Vollenweider betreffe...,661,661,661,2016-08-24
3,2631,Kleine Anfrage von Philip C. Brunner betreffen...,558,558,558,2017-04-16
4,2449,Interpellation der SP-Fraktion betreffend Steu...,404,404,404,2014-11-13
...,...,...,...,...,...,...
494,810,Motion von Marcel Meyer betreffend Vergabe von...,999,999,999,2002-08-13
495,801,Motion von Hans Abicht betreffend Raumkonzept ...,787,787,787,2006-08-22
496,666,Motion der Kommission Teilrevision Personalges...,981,981,981,2002-06-06
497,304,Motion von Manuela Weichelt betreffend HIV-Prä...,973,973,973,2003-05-27


In [20]:
df_np["VorlageNR"]=df_np["VorlageNR"].astype(str)

In [21]:
df_alles=pd.merge(df_alles, df_np, how="left", on="VorlageNR")
df_alles

,VorlageNR,Geschäft,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Einreichedatum,Namenganz,P,Stand,Zuständigkeit,Art des Geschäfts,Status,url
0,2932,"Kleine Anfrage von Laura Dittli, Fabio Iten un...",707,707,707,2019-02-08,Fabio Iten,CVP,Antwort des Regierungsrats,Baudirektion,Kleine Anfrage,abgeschlossen,/gast/geschaefte/1920
1,2932,"Kleine Anfrage von Laura Dittli, Fabio Iten un...",707,707,707,2019-02-08,Laura Dittli,CVP,Antwort des Regierungsrats,Baudirektion,Kleine Anfrage,abgeschlossen,/gast/geschaefte/1920
2,2932,"Kleine Anfrage von Laura Dittli, Fabio Iten un...",707,707,707,2019-02-08,Patrick Iten,CVP,Antwort des Regierungsrats,Baudirektion,Kleine Anfrage,abgeschlossen,/gast/geschaefte/1920
3,2919,Interpellation der SP-Fraktion betreffend was ...,696,696,696,NaN,SP-Fraktion,SP,Kenntnisnahme,Sicherheitsdirektion,Interpellation,abgeschlossen,/gast/geschaefte/1907
4,2858,Interpellation von Willi Vollenweider betreffe...,661,661,661,2016-08-24,Willi Vollenweider,pl,Kenntnisnahme,Sicherheitsdirektion,Interpellation,abgeschlossen,/gast/geschaefte/1848
...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,810,Motion von Marcel Meyer betreffend Vergabe von...,999,999,999,2002-08-13,Marcel Meyer,CVP,Datenmigration,NaN,Motion,abgeschlossen,/gast/geschaefte/1335
691,801,Motion von Hans Abicht betreffend Raumkonzept ...,787,787,787,2006-08-22,Hans Abicht,FDP,Datenmigration,NaN,Motion,abgeschlossen,/gast/geschaefte/1333
692,666,Motion der Kommission Teilrevision Personalges...,981,981,981,2002-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
693,304,Motion von Manuela Weichelt betreffend HIV-Prä...,973,973,973,2003-05-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
del df_alles["Unnamed: 0"]
del df_alles["Unnamed: 0.1"]
del df_alles ["Unnamed: 0.1.1"]


In [25]:
df_alles
# Irgendwas lief schief!!!!! ich habe viel zu wenig Reihen und WEichelt ist nicht drin und auch sonst fehlen daten.


,VorlageNR,Geschäft,Einreichedatum,Namenganz,P,Stand,Zuständigkeit,Art des Geschäfts,Status,url
0,2932,"Kleine Anfrage von Laura Dittli, Fabio Iten un...",2019-02-08,Fabio Iten,CVP,Antwort des Regierungsrats,Baudirektion,Kleine Anfrage,abgeschlossen,/gast/geschaefte/1920
1,2932,"Kleine Anfrage von Laura Dittli, Fabio Iten un...",2019-02-08,Laura Dittli,CVP,Antwort des Regierungsrats,Baudirektion,Kleine Anfrage,abgeschlossen,/gast/geschaefte/1920
2,2932,"Kleine Anfrage von Laura Dittli, Fabio Iten un...",2019-02-08,Patrick Iten,CVP,Antwort des Regierungsrats,Baudirektion,Kleine Anfrage,abgeschlossen,/gast/geschaefte/1920
3,2919,Interpellation der SP-Fraktion betreffend was ...,NaN,SP-Fraktion,SP,Kenntnisnahme,Sicherheitsdirektion,Interpellation,abgeschlossen,/gast/geschaefte/1907
4,2858,Interpellation von Willi Vollenweider betreffe...,2016-08-24,Willi Vollenweider,pl,Kenntnisnahme,Sicherheitsdirektion,Interpellation,abgeschlossen,/gast/geschaefte/1848
...,...,...,...,...,...,...,...,...,...,...
690,810,Motion von Marcel Meyer betreffend Vergabe von...,2002-08-13,Marcel Meyer,CVP,Datenmigration,NaN,Motion,abgeschlossen,/gast/geschaefte/1335
691,801,Motion von Hans Abicht betreffend Raumkonzept ...,2006-08-22,Hans Abicht,FDP,Datenmigration,NaN,Motion,abgeschlossen,/gast/geschaefte/1333
692,666,Motion der Kommission Teilrevision Personalges...,2002-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
693,304,Motion von Manuela Weichelt betreffend HIV-Prä...,2003-05-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_alles # Irgendwie heats ein Durenand gegeben. Muss ich ein ander Mal anschauen

In [ ]:
df_alles

# XX Zum Resultat hier mal das Dataframe der Hauptseite sowie der Unterseiten zusammenfügen.

In [ ]:
df_mit_dat=pd.merge(df_a, df_g, how="left", on="GeschäftsNR") #ich verbinde die beiden DataFrames

In [ ]:
#nun will ich noch das Datum zum Idex machen
df_mit_dat.set_index("Einreichedatum", inplace=True)

In [ ]:
df_mit_dat.dtypes

# Ein erstes Ergebnis = ab 2013 alle  Vorstösse
# ACHTUNG das stimmt ja so gar nicht, da bei mehreren Personen
# auch mehrere Einreichungen verzeichnet werden!!!

In [ ]:
#Nun will ich ein erstes Mal in die DAten schauen, zumindest ab 2013 - ob es ein Muster beim Einreichen gibt.
#DAzu krame ich jetzt alle Vorstösse raus

df_vo_all=df_mit_dat[{"Art des Geschäfts" : ["Motion","Interpellation","Postulat","Kleine Anfrage"] }]

In [ ]:
df_mit_dat["Art des Geschäfts"]["GeschäftsNR"]

In [ ]:
df_vo_all["2013":].resample("SMS").count().plot(figsize=(8,4))

# Regex Baby! 
### Ich versuche mit Regex in Pandas an die Namen zu kommen

In [ ]:
#nun versuche ich mit Regex an die einzelnen Namen zu kommen.
regex_n=r"\bvon(.*und.\w*.\w*\b)"
regex_n2= r"^(\w*.\w*\b)\,.(\w*.\w*\b)"

In [ ]:
df_n1=df_a['Geschäft'].str.extract(regex_n)

In [ ]:
df_n1